In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# x => [공부 시간, 과외 시간]
x = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열
print(x)
print(type(x))

xData = np.array(x)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# y => [합격 여부] => 실제값
y = [0, 0, 0, 1, 1, 1, 1] # 1행 7열
print(y)
print(type(y))

yData = np.array(y)
print(yData)
print(type(yData))

# reshape(): 넘파이에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 넘파이 배열 => 7행 1열 넘파이 배열
yData = np.array(y).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 0, 1, 1]
<class 'list'>
[0 0 0 1 0 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입될 넘파이 배열의 차원을 지정해야 한다.
# [None, 2] => placeholder에 대입될 넘파이 배열의 행의 개수는 몇개라도 상관없고 열의 개수는 무조건 2개라는 의미이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치와 y절편을 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float32)) # 난수를 2행 1렬로 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32))
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')
print('a1: {:f}, a2: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))
print('a1: %f, a2: %f, b: %f' % (sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a
[[0.6227963 ]
 [0.45350718]]
b
[0.6257187]
a1: 0.622796, a2: 0.453507, b: 0.625719
a1: 0.622796, a2: 0.453507, b: 0.625719


시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# sigmoid(): tensorflow에서 시그모이드 방정식을 계산한다.
# matmul(): tensorflow에서 행렬의 곱(적)을 계산한다.
# Y = 1 / (1 + np.e ** -(a1 * x1 + a2 * x2 + b))
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식, 예측값
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강

sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 -1을 리턴시킨다.

In [7]:
sess = tf.Session()
# cast(data, dtype): tensorflow에서 데이터(data)를 dtype으로 형변환 한다.
predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
print(sess.run(predict)) # [1 2]
# cast() 메소드는 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환 한다.
predict = tf.cast(tf.constant([True, False]), dtype=tf.int32)
print(sess.run(predict)) # [1 0]
predict = tf.cast([0.5 >= 0.5, 0.5 < 0.5], dtype=tf.int32)
print(sess.run(predict))

[1 2]
[1 0]
[1 0]


sigmoid() 메소드를 실행한 예측값(0 또는 1)을 계산한다.

In [8]:
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [9]:
sess = tf.Session()
# equal(): tensorflow에서 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0))) # True False
# equal() 메소드로 예측값과 실제값이 같은가 비교 후 True 또는 False를 cast() 메소드로 1 또는 0으로 캐스팅하고 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float32))

True False


학습 시킨다.

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # sess.run([실행할 내용, ...], feed_dict={플레이스홀더 이름: 플레이스홀더에 저장할 데이터, ...})
    a_, b_, loss_, _, predict_, accuracy_ = sess.run([a, b, loss, gradient_descent, predict, accuracy], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:6d}, loss: {:7.4f}, a1: {:7.4f}, a2: {:7.4f}, b: {:7.4f}'.format(epoch, loss_, a_[0][0], a_[1][0], b_[0]))
        # print('epoch: %6d, loss: %7.4f, a: %7.4f, b: %7.4f' % (epoch, sess.run(loss), sess.run(a), sess.run(b)))
print('실제값\n', yData, sep='')
print('예상값\n', predict_, sep='')
print('정확도: {:6.2%}'.format(accuracy_))

epoch:      0, loss:  3.0453, a1:  0.3446, a2:  0.1460, b:  0.0106
epoch:    300, loss:  0.4279, a1:  0.8477, a2: -0.7775, b: -2.3259
epoch:    600, loss:  0.3921, a1:  0.7616, a2: -0.5583, b: -3.2395
epoch:    900, loss:  0.3754, a1:  0.6306, a2: -0.2727, b: -3.8718
epoch:   1200, loss:  0.3642, a1:  0.4926, a2:  0.0005, b: -4.3609
epoch:   1500, loss:  0.3563, a1:  0.3650, a2:  0.2450, b: -4.7634
epoch:   1800, loss:  0.3504, a1:  0.2521, a2:  0.4593, b: -5.1078
epoch:   2100, loss:  0.3458, a1:  0.1536, a2:  0.6461, b: -5.4097
epoch:   2400, loss:  0.3423, a1:  0.0676, a2:  0.8098, b: -5.6787
epoch:   2700, loss:  0.3394, a1: -0.0080, a2:  0.9543, b: -5.9211
epoch:   3000, loss:  0.3371, a1: -0.0750, a2:  1.0832, b: -6.1414
실제값
[[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
예상값
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]
정확도: 71.43%
